In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
%pip install datasets transformers sacrebleu torch sentencepiece "transformers[sentencepiece]" huggingface_hub wandb hydra-core

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 1.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

root_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
print(root_dir)
assert os.path.exists(root_dir), f'Could not find root directory at {root_dir}'
sys.path.insert(0, root_dir)

from custom_utils.config_handler import read_config, pprint_config

/home/jovyan/rodion/other/trans


In [3]:
overrides = ["setup=finetune"]
cfg = read_config(overrides=overrides)
pprint_config(cfg)

{
  "root": "/home/jovyan/rodion/other/trans",
  "dataset": {
    "path": "waleko/unarXive-en2ru"
  },
  "params": {
    "batch_size": 16,
    "max_length": 512,
    "train_args": {
      "evaluation_strategy": "epoch",
      "learning_rate": 2e-05,
      "per_device_train_batch_size": 16,
      "per_device_eval_batch_size": 16,
      "weight_decay": 0.01,
      "save_total_limit": 3,
      "num_train_epochs": 4,
      "predict_with_generate": true
    },
    "wandb_args": {
      "report_to": "wandb",
      "run_name": "finetune"
    }
  },
  "pretrained": {
    "name": "opus-en-ru",
    "model_and_tokenizer_name": "Helsinki-NLP/opus-mt-en-ru"
  },
  "finetuned": {
    "name": "opus-distilled-en-ru",
    "model_and_tokenizer_name": "under-tree/transformer-en-ru",
    "output_dir": "/home/jovyan/rodion/other/trans/models/opus-distilled-en-ru/finetuned"
  }
}


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from huggingface_hub import HfApi

# Create an instance of HfApi
api = HfApi()

# Check if logged in and get username
try:
    user_info = api.whoami()
    username = user_info['name']
    print(f"Logged in to Hugging Face as: {username}")
except Exception as e:
    print(e)
    print("Not logged in to Hugging Face.")


Logged in to Hugging Face as: under-tree


In [6]:
import wandb
wandb.login(relogin=True)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/user/.netrc


True

In [7]:
wandb.login()

wandb: Currently logged in as: rodionkhvorostov (wide-learning). Use `wandb login --relogin` to force relogin


True

In [8]:
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=true
%env WANDB_NOTEBOOK=finetune

env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=true
env: WANDB_NOTEBOOK=finetune


In [9]:
import transformers
print(transformers.__version__)

4.35.0.dev0


In [10]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset(cfg.dataset.path)
metric = load_metric("sacrebleu")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/tmp/ipykernel_341/1892407716.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 9082
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 702
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 568
    })
})

In [12]:
import numpy as np
n = 3
rnd_idx = np.random.randint(0, len(raw_datasets["train"]), n)
raw_datasets["train"][rnd_idx]

{'source': ['\nA parametric approach requires assuming the form of the distribution with unknown parameters. The method of moments or the maximum likelihood methods are then applied to obtain the estimations of the unknown parameters.\n\nA non-parametric methods such as the kernel methods ([1]}, [2]}, [3]}, [4]}) and the empirical distribution (i.e., \\(F_n\\) ) avoid any assumption about the form of the distribution.\n\nA combination of both approaches can lead to a semi-parametric methods.\n\n',
  'The Bardeen solution has motivated deeper works about singularity avoidance may be realized generally. Several other researchers paid attention to theories of gravity coupled to nonlinear electrodynamics, and proposed other solutions in different contexts. Some solutions that are relevant to this work are analyzed in Refs. [1]}, [2]}, [3]}, [4]}, [5]} and Refs. [6]}, [7]}.\n',
  'In typical implementations, one possibility is to exploit the structure of the underlying PDE to set many of th

In [13]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained.model_and_tokenizer_name)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.pretrained.model_and_tokenizer_name)

The reason for such tokenization difference (as I think) is that actually for ru, and for en are used different tokenizers

In [15]:
example_sample = ["Привет, мир!"]
tokens_usual = tokenizer(example_sample)
with tokenizer.as_target_tokenizer():
    tokens_target = tokenizer(example_sample)

print("Tokens usual:", tokens_usual)
print("Tokens target:", tokens_target)

Tokens usual: {'input_ids': [[21, 2408, 779, 53, 222, 30, 275, 2, 21, 46, 53, 779, 56, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
Tokens target: {'input_ids': [[543, 1877, 2, 1920, 56, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3854: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [16]:
max_length = cfg.params.max_length

# remove
def preprocess_function(examples):
    inputs = [ex for ex in examples["source"]]
    targets = [ex for ex in examples["target"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

In [17]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[156, 1747, 4, 2877, 5, 338, 791, 289, 35659, 592, 2168, 11, 1491, 20, 26007, 4, 18426, 21, 58925, 1728, 2, 4657, 8220, 25, 542, 42118, 35409, 237, 12281, 2, 1113, 37966, 2293, 43511, 237, 12281, 2, 29, 11, 17992, 2637, 6786, 49308, 237, 12281, 2, 1113, 37966, 2293, 11, 2088, 9318, 202, 1565, 237, 12281, 2, 8, 1220, 11, 1291, 27328, 50881, 237, 12281, 3, 30276, 188, 4, 57893, 127, 36, 1179, 10, 4, 23571, 3, 13988, 2, 105, 12905, 4, 29850, 5812, 5, 338, 34254, 23, 2, 3758, 39, 4, 1133, 5, 310, 31483, 8, 4, 1717, 1880, 5, 5630, 1133, 2, 201, 77, 13, 32154, 1969, 1747, 5, 338, 289, 35659, 592, 2168, 11, 1491, 3, 0], [2470, 35588, 118, 29050, 23, 20, 21, 16779, 1145, 91, 57655, 39634, 1114, 11, 166, 12281, 16779, 24, 10, 14, 7825, 2088, 21, 117, 105, 5719, 4, 542, 9665, 592, 2526, 14236, 23, 20, 4, 1221, 17240, 1969, 8416, 16681, 3876, 104, 5967, 23, 2, 81, 55, 13, 2506, 427, 58437, 5, 27170, 14, 47763, 24, 10, 35409, 237, 12281, 64, 27170, 17408, 10073, 10, 43511, 237, 1228

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=["source", "target"])

Map:   0%|          | 0/9082 [00:00<?, ? examples/s]

Map:   0%|          | 0/702 [00:00<?, ? examples/s]

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

## Finetuning

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.pretrained.model_and_tokenizer_name)

In [21]:
pprint_config(cfg.params.train_args)
pprint_config(cfg.params.wandb_args)

{
  "evaluation_strategy": "epoch",
  "learning_rate": 2e-05,
  "per_device_train_batch_size": 16,
  "per_device_eval_batch_size": 16,
  "weight_decay": 0.01,
  "save_total_limit": 3,
  "num_train_epochs": 4,
  "predict_with_generate": true
}
{
  "report_to": "wandb",
  "run_name": "finetune"
}


In [22]:
model_name = cfg.pretrained.name
args = Seq2SeqTrainingArguments(
    cfg.finetuned.output_dir,
    **cfg.params.train_args,
    **cfg.params.wandb_args,
    push_to_hub=True,
    hub_model_id=cfg.finetuned.model_and_tokenizer_name
)

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [24]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # The labels value for the text that is not being predicted gets set to -100. 
    # Replace these values with pad
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [25]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.941200,0.653884,67.599700,127.287700
2,0.697700,0.608778,69.201200,127.783500
3,0.631200,0.589383,69.907900,127.373200
4,0.587600,0.584227,70.363300,126.859000


TrainOutput(global_step=2272, training_loss=0.6974154055958063, metrics={'train_runtime': 1218.4183, 'train_samples_per_second': 29.816, 'train_steps_per_second': 1.865, 'total_flos': 2196400063905792.0, 'train_loss': 0.6974154055958063, 'epoch': 4.0})

In [27]:
print("Perplexity:", np.exp(trainer.evaluate()["eval_loss"]))

Perplexity: 1.7936033980595856


In [28]:
import json
test_score = trainer.evaluate(tokenized_datasets["test"])

print(json.dumps(test_score, indent=4))

{
    "eval_loss": 0.6561718583106995,
    "eval_bleu": 67.197,
    "eval_gen_len": 126.7394,
    "eval_runtime": 194.0045,
    "eval_samples_per_second": 2.928,
    "eval_steps_per_second": 0.186,
    "epoch": 4.0
}


In [29]:
wandb.finish()

eval/bleu,▂▅▇██▁
eval/gen_len,▅█▅▂▂▁
eval/loss,█▃▂▁▁█
eval/runtime,██▇▇▇▁
eval/samples_per_second,▁▁▆███
eval/steps_per_second,▁▁▅▆▆█
train/epoch,▁▁▃▄▅▆▇████
train/global_step,▁▁▃▄▅▆▇████
train/learning_rate,█▆▃▁
train/loss,█▃▂▁
train/total_flos,▁


In [30]:
import os
for dirname, _, filenames in os.walk(model_name):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [31]:
from transformers import MarianMTModel, MarianTokenizer

path = cfg.finetuned.model_and_tokenizer_name  # os.path.join(cfg.finetuned.output_dir, "checkpoint-2000")
tokenizer = MarianTokenizer.from_pretrained(path)
model = MarianMTModel.from_pretrained(path)

In [32]:
sentence_from_paper = "Transformers are slow and memory-hungry on long sequences, since the time and memory complexity of self-attention are quadratic in sequence length. Approximate attention methods have attempted to address this problem by trading off model quality to reduce the compute complexity, but often do not achieve wall-clock speedup. "
src_text = [sentence_from_paper]
tokenized_text = tokenizer(src_text, return_tensors="pt", padding=True)
translated = model.generate(**tokenized_text)
decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)

In [33]:
from pprint import pprint

for src, tran in zip(src_text, decoded):
    print("Source")
    pprint(src)
    print("Target")
    pprint(tran)

Source
('Transformers are slow and memory-hungry on long sequences, since the time '
 'and memory complexity of self-attention are quadratic in sequence length. '
 'Approximate attention methods have attempted to address this problem by '
 'trading off model quality to reduce the compute complexity, but often do not '
 'achieve wall-clock speedup. ')
Target
('Трансформеры медленны и страдают памятью на длинных последовательностях, так '
 'как время и сложность памяти самовнимания квадратичны по длине '
 'последовательности. Приближающиеся методы внимания пытались решить эту '
 'проблему, отменяя качество модели, чтобы снизить сложность вычислений, но '
 'часто не достигают ускорения на стене.')
